In [4]:
from collections import deque

# 1. Read junction boxes
points = []
with open("input,txt") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        x, y, z = map(int, line.split(","))
        points.append((x, y, z))

n = len(points)
print("Number of junction boxes:", n)

# 2. Build all pair distances (squared Euclidean)
edges = []
for i in range(n):
    x1, y1, z1 = points[i]
    for j in range(i + 1, n):
        x2, y2, z2 = points[j]
        d2 = (x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2
        edges.append((d2, i, j))

# 3. Sort by distance
edges.sort(key=lambda e: e[0])

# 4. Add the 1000 shortest edges, skipping only duplicates
adj = [set() for _ in range(n)]
connections = 0

for d2, i, j in edges:
    if connections >= 1000:
        break
    if j in adj[i]:
        # already directly connected; skip this pair
        continue
    adj[i].add(j)
    adj[j].add(i)
    connections += 1

print("Connections made:", connections)

# 5. Find connected components (circuits) via BFS
visited = [False] * n
sizes = []

for start in range(n):
    if visited[start]:
        continue
    q = deque([start])
    visited[start] = True
    size = 0
    while q:
        v = q.popleft()
        size += 1
        for nb in adj[v]:
            if not visited[nb]:
                visited[nb] = True
                q.append(nb)
    sizes.append(size)

sizes.sort(reverse=True)
print("Circuit sizes (largest first):", sizes[:10])

# 6. Product of the three largest circuits
if len(sizes) >= 3:
    result = sizes[0] * sizes[1] * sizes[2]
elif len(sizes) == 2:
    result = sizes[0] * sizes[1] * 1
elif len(sizes) == 1:
    result = sizes[0] ** 3
else:
    result = 0

print("Answer:", result)


Number of junction boxes: 1000
Connections made: 1000
Circuit sizes (largest first): [54, 51, 45, 44, 29, 24, 23, 19, 18, 17]
Answer: 123930


In [6]:
# Part Two solution for Day 8-style puzzle

# 1. Read junction box coordinates from input.txt
points = []
with open("input,txt") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        x, y, z = map(int, line.split(","))
        points.append((x, y, z))

n = len(points)
print("Number of junction boxes:", n)

# 2. Build all pairwise edges with squared Euclidean distance
edges = []
for i in range(n):
    x1, y1, z1 = points[i]
    for j in range(i + 1, n):
        x2, y2, z2 = points[j]
        d2 = (x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2
        edges.append((d2, i, j))

# 3. Sort edges from closest to farthest
edges.sort(key=lambda e: e[0])

# 4. Union–find to track circuits
parent = list(range(n))
rank = [0] * n

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra == rb:
        return False
    if rank[ra] < rank[rb]:
        ra, rb = rb, ra
    parent[rb] = ra
    if rank[ra] == rank[rb]:
        rank[ra] += 1
    return True

# 5. Connect closest pairs until everything is in one circuit
components = n
last_edge = None

for d2, i, j in edges:
    if union(i, j):
        components -= 1
        last_edge = (i, j)
        if components == 1:
            break

print("Components after process:", components)
print("Last edge indices:", last_edge)

# 6. Compute the product of X coordinates of the last two boxes
x1, _, _ = points[last_edge[0]]
x2, _, _ = points[last_edge[1]]
product = x1 * x2

print("Last X coordinates:", x1, x2)
print("Answer (product):", product)


Number of junction boxes: 1000
Components after process: 1
Last edge indices: (189, 653)
Last X coordinates: 2496 10953
Answer (product): 27338688
